In [1]:
# Cell 1: 라이브러리 임포트 및 설정
import pandas as pd
from est_utils import BACKBONE_CLASSES, split_soft_label_data

# 설정
INPUT_CSV_PATH = '04_softlabel_dataset.csv'
OUTPUT_CSV_PATH = '06_softlabel_dataset_resplit.csv'

soft_label_columns = [f'soft_{cls}' for cls in BACKBONE_CLASSES]

# 분할 매개변수
TEST_SIZE = 0.2 # 전체 데이터 대비 Test 세트 비율
VAL_SIZE = 0.5 # Test 세트 분리 후 남은 데이터 대비 Validation 세트 비율 (0.5면 50%)
RANDOM_STATE = 42 # 재현성을 위한 랜덤 시드
RARE_THRESHOLD = 10 # 계층적 분할 시 희귀 클래스로 간주할 최소 샘플 수

In [2]:
# Cell 2: 데이터 로드
try:
    df = pd.read_csv(INPUT_CSV_PATH)
    print(f"데이터 로드 성공: {INPUT_CSV_PATH}, {len(df)} 행")
except FileNotFoundError:
    print(f"오류: {INPUT_CSV_PATH} 파일을 찾을 수 없습니다. 04_soft_labeling_processor.ipynb를 먼저 실행하여 파일을 생성하세요.")
    df = pd.DataFrame() # 빈 데이터프레임으로 초기화
except Exception as e:
    print(f"데이터 로드 중 오류 발생: {e}")
    df = pd.DataFrame()

데이터 로드 성공: 04_softlabel_dataset.csv, 8394 행


In [3]:
# Cell 3: 데이터 분할 수행
if not df.empty:
    train_df, val_df, test_df = split_soft_label_data(
        df,
        soft_label_columns,
        test_size=TEST_SIZE,
        val_size=VAL_SIZE,
        random_state=RANDOM_STATE,
        rare_threshold=RARE_THRESHOLD
    )
    print("데이터 분할 완료.")
else:
    train_df, val_df, test_df = pd.DataFrame(), pd.DataFrame(), pd.DataFrame()
    print("데이터프레임이 비어있어 분할을 수행할 수 없습니다.")

데이터 분할 완료.


In [4]:
# Cell 4: 분할 통계 표시
if not train_df.empty or not val_df.empty or not test_df.empty:
    print(f'전체 데이터: {len(df)}개')
    print(f'Train 세트: {len(train_df)}개')
    print(f'Validation 세트: {len(val_df)}개')
    print(f'Test 세트: {len(test_df)}개')

    # 분할된 세트의 'stratify_key' 분포 확인 (get_stratify_key는 split_soft_label_data 내부에서 호출됨)
    # df에 'stratify_key' 컬럼이 추가되어 있어야 함
    if 'stratify_key' in df.columns:
        train_distribution = train_df['stratify_key'].value_counts()
        train_distribution_norm = train_df['stratify_key'].value_counts(normalize=True)

        val_distribution = val_df['stratify_key'].value_counts()
        val_distribution_norm = val_df['stratify_key'].value_counts(normalize=True)

        test_distribution = test_df['stratify_key'].value_counts()
        test_distribution_norm = test_df['stratify_key'].value_counts(normalize=True)

        # 결측값(NaN)은 0으로 채우고, 인덱스(감정)를 기준으로 정렬
        distribution_df = pd.concat(
            [train_distribution, val_distribution, test_distribution, train_distribution_norm, val_distribution_norm, test_distribution_norm],
            axis=1,
            keys=['Train', 'Val', 'Test', 'Train (Norm)', 'Val (Norm)', 'Test (Norm)']
        ).fillna(0).astype(float)

        print("\n데이터 세트별 'stratify_key' 감정 분포:")
        print(distribution_df.to_markdown())
    else:
        print("'stratify_key' 컬럼이 데이터프레임에 없어 분포를 표시할 수 없습니다.")
else:
    print("분할된 데이터프레임이 비어있어 통계를 표시할 수 없습니다.")

전체 데이터: 8394개
Train 세트: 6716개
Validation 세트: 839개
Test 세트: 839개

데이터 세트별 'stratify_key' 감정 분포:
| stratify_key       |   Train |   Val |   Test |   Train (Norm) |   Val (Norm) |   Test (Norm) |
|:-------------------|--------:|------:|-------:|---------------:|-------------:|--------------:|
| happy              |    1727 |   216 |    216 |    0.257147    |   0.257449   |    0.257449   |
| sad                |    1446 |   181 |    181 |    0.215307    |   0.215733   |    0.215733   |
| anger              |    1334 |   166 |    167 |    0.19863     |   0.197855   |    0.199046   |
| surprise           |     990 |   124 |    124 |    0.147409    |   0.147795   |    0.147795   |
| fear               |     606 |    76 |     76 |    0.0902323   |   0.090584   |    0.090584   |
| neutral            |     549 |    69 |     68 |    0.0817451   |   0.0822408  |    0.0810489  |
| happy-surprise     |      45 |     5 |      6 |    0.00670042  |   0.00595948 |    0.00715137 |
| fear-happy         | 

In [5]:
# Cell 5: 분할 데이터 저장
if not df.empty:
    # 원본 DataFrame에 새로운 phase 할당
    df.loc[train_df.index, 'new_phase'] = 'train'
    df.loc[val_df.index, 'new_phase'] = 'val'
    df.loc[test_df.index, 'new_phase'] = 'test'

    # 'new_phase'로 'phase' 덮어쓰기
    df['phase'] = df['new_phase']

    # 'new_phase' 컬럼 삭제
    df.drop(columns=['new_phase'], inplace=True)

    df.to_csv(OUTPUT_CSV_PATH, index=False)
    print(f"분할 결과가 {OUTPUT_CSV_PATH} 파일로 저장되었습니다.")
    display(df.head())
else:
    print("저장할 데이터프레임이 비어있습니다.")

분할 결과가 06_softlabel_dataset_resplit.csv 파일로 저장되었습니다.


,phase,category,filename,img_path,exists,annot_A,annot_B,annot_C,soft_anger,soft_contempt,soft_disgust,soft_fear,soft_happy,soft_neutral,soft_sad,soft_surprise,stratify_key
0,train,anger,6oj439e3fbcc52759fb3093035b7c0ecc55c93543dae63...,./Data/EST_data/img_train/anger/6oj439e3fbcc52...,True,분노,분노,분노,0.800000,0.0,0.0,0.000000,0.000000,0.200000,0.000000,0.0,anger
1,val,anger,65rsfe402042f34319e10128c1ab9614e2f967690a64a0...,./Data/EST_data/img_train/anger/65rsfe402042f3...,True,분노,분노,분노,0.800000,0.0,0.0,0.000000,0.000000,0.200000,0.000000,0.0,anger
2,train,anger,b1cbe34734870cc11c33334e02bea93ac3a3b061caab62...,./Data/EST_data/img_train/anger/b1cbe34734870c...,True,상처,불안,슬픔,0.000000,0.0,0.0,0.333333,0.000000,0.200000,0.466667,0.0,sad
3,train,anger,llfycc0aa29599cc63cace3610fdaaad3a99aab2ee38c9...,./Data/EST_data/img_train/anger/llfycc0aa29599...,True,분노,분노,분노,0.800000,0.0,0.0,0.000000,0.000000,0.200000,0.000000,0.0,anger
4,train,anger,3hww73b70615461a7336d0383b53582f8bf804f6e0f30d...,./Data/EST_data/img_train/anger/3hww73b7061546...,True,기쁨,분노,분노,0.533333,0.0,0.0,0.000000,0.333333,0.133333,0.000000,0.0,anger
